# Advanced Python Concepts

In this tutorial, a few advanced concepts (not class related) are introduced. This includes

- packing and unpacking
- context manager
- decorator and factories

In [1]:
import time

## Packing and unpacking of values

Using `*` or `**` we can pack/unpack list-like objects and dict-like objects, respectively.

Let's play around...

In [2]:
a, c, *b = [3, 4, 4.5, 5, 6]

In [3]:
b

[4.5, 5, 6]

In [4]:
a = [3, 4, 5]

In [5]:
d, e, f, g, h, i = *a, *b

Now we should be able to understand the `*args` and `**kwargs` for functions. Let's look at it:

In [6]:
def func(*args, **kwargs):
    print(f'args are {args}')
    print(f"kwargs are {kwargs}")

In [7]:
mykwargs = {'a': 5, 'b': 3}
myargs = [1, 3, 4]
func(*myargs, *mykwargs)

args are (1, 3, 4, 'a', 'b')
kwargs are {}


In [8]:
func(5, a=4)

args are (5,)
kwargs are {'a': 4}


In [9]:
# play around with it!

## Context manager

A context manager is an object that responds to a `with` statement. It may returns something. The basic idea is that some action is performed when entering a context and again when exiting it.

```
with context as var:
    # do something
```
translates to
```
# execute context entering code
var = return_from_context_entering_code
# do something
# execute context leaving code
```

The great advantage here is that the "leaving code" is automatically executed whenever we step out of the context!

This proved to be incredibly useful when operations have cleanup code that we need to execute yet that is tedious to write manually and can be forgotten.

### Using `yield`

One way to create a context manager is to have a function that has a `yield`.

_What is `yield`?_: It's like a return, except that the executioin stops at the `yield`, let's other code execute and, at some point, **continues** again where the yield was. Examples are:
- iterator: a function that yields elements. Everytime it is called, it is supposed to yield an element and then continue from there
- asynchronous programing: it stops and waits until something else is finished
- in the context manager, as we will see

In [10]:
import contextlib

@contextlib.contextmanager
def printer(x):
    print(f'we just entered the context manager and will yield {x}')
    yield x
    print(f'Finishing the context manager, exiting')

In [11]:
with printer(5) as number:
    print(f"we're inside, with number={number}")
print("left manager")

we just entered the context manager and will yield 5
we're inside, with number=5
Finishing the context manager, exiting
left manager


#### Where is this useful

Basically with stateful objects. This includes anything that can be set and changed (mutable objects).

In [12]:
with open('tmp.txt', 'w') as textfile:
    textfile.write('asdf')

The implementation roughly looks like this:

In [13]:
import contextlib

@contextlib.contextmanager
def myopen(f, mode):
    opened = open(f, mode)
    yield opened
    opened.close()
    

**Exercise**: create a context manager that _temporarily_ sets a `'value'` key to 42 of a dict and switches it back to the old value on exit

In [14]:
testdict = {'value': 11, 'name': 'the answer'}

to be invoked like this

```python
with manager(testdict) as obj:
    # here the value is 42
# here the value is 11
```

In [15]:
# SOLUTION
@contextlib.contextmanager
def func(x):
    yield x

with func(5) as var1:
    print('inside')
print(var1)

inside
5


In [16]:
@contextlib.contextmanager
def set_answer(obj):
    old_value = obj.get('value')
    obj['value'] = 42
    yield obj
    obj['value'] = old_value

## Using a class

Instead of using the `yield`, we can have advanced control over the enter and exit methods by creating a class and implementing the two methods `__enter__` and `__exit__`

In [17]:
class MyContext:
    
    def __init__(self, x):
        self.x = x
    
    def __enter__(self):
        x = self.x
        print('entered')
        return x ** 2
    
    def __exit__(self, type_, value, traceback):  # but let's not go into things in detail here
        self.x = 42
        print('exited')

In [18]:
with MyContext(5) as x:
    print(x)

entered
25
exited


While a class is way more powerful and offers ways to catch exceptions and more in the exit, ususally the functional way is enough and should then be preferred. If it doesn't give you enough flexibility, remember the class, look it up and figure out all the things needed.

## Decorators and factories

Sometimes we can't write a function fully by hand but want to create it programatically. This pattern is called a "factory". To achieve this, instead of having a function that returns an integer (an object), a list (an object), a dict (an object) or an array (an object), we return a function (an object). We see that the concept of Python, "everything is an object", starts being very useful here.

In [19]:
def make_power_func(power):
    def func(x):
        return x ** power
    return func

In [20]:
pow3 = make_power_func(3)

In [21]:
pow3(2)

8

In [22]:
def make_power_func(power):
    def func(x):
        return x ** power
    power = 42
    return func

In [23]:
pow3 = make_power_func(3)

In [24]:
pow3(2)

4398046511104

In [25]:
# Exercise: test it here

Another example is to create a timing wrapper. **Exercise**: create a timing function that can be used as follows

```
timed_pow3 = fime_func(pow3)
pow3(...)
```

HINT, scetch of solution
```python
def time_func(func):
    def new_func(...):
        print('start')
        func(...)
        print('stop')
    return new_func
```

In [26]:
# SOLUTION
def timed_func(func):
    def wrapped_func(*args, **kwargs):
        print(args)
        print(kwargs)
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        print(f'time needed: {end - start}')
    return wrapped_func

In [27]:
def add_notime(x, y):
    return x + y

In [28]:
add_timed = timed_func(add_notime)

In [29]:
import time

In [30]:
add_timed(y=4, x=5)

()
{'y': 4, 'x': 5}
time needed: 7.152557373046875e-07


In [31]:
# test it here

## Decorator

There is another way, just syntactical sugar, to make this automatic: a decorator. It is invoked as below

In [32]:
@timed_func
def add(x, y):
    return x + y

Again, as for the contextmanager, we can also use a class here to give more flexibility and create a decorator that takes _arguments_.

# Exceptions

In [34]:
raise TypeError("Has to be int, not str")

TypeError: Has to be int, not str

In [35]:
class MyError(Exception):
    pass

In [36]:
raise MyError("Hello world")

MyError: Hello world

In [37]:
try:
    raise TypeError
except:  # any exception
    pass

In [45]:
try:
    raise TypeError("Type was wrong, unfortunately")
except TypeError as error:  # any exception
    print(f'caught Type {error}')
    raise
except ValueError as error:
    print(f'caugth Value: {error}')

caught Type Type was wrong, unfortunately


TypeError: Type was wrong, unfortunately

In [49]:
try:
    print('no error raised')
#     raise TypeError("Type was wrong, unfortunately")
except TypeError as error:  # any exception
    print(f'caught Type {error}')
except ValueError as error:
    print(f'caugth Value: {error}')
else:
    print("No error")
    
print("Executed after block")

no error raised
No error
Executed when passed


In [52]:
try:
#     pass
#     raise TypeError("Type was wrong, unfortunately")
    raise RuntimeError("Type was wrong, unfortunately")
except TypeError as error:  # any exception
    print(f'caught Type {error}')
except ValueError as error:
    print(f'caugth Value: {error}')
else:
    print("No error")
finally:  # POWERFUL! Guarantied to be executed
    print('Finally run')
print("Executed when passed")

Finally run


RuntimeError: Type was wrong, unfortunately

In [68]:
def func(x):
    try:
        if x == 5:
            raise RuntimeError('called inside func')
    except RuntimeError as error:
        return error
    else:
        print('else before 42')
        return 42
        print('after else 42')
    finally:
        print("cleaned up")
        return 11
    

In [69]:
func(6)

else before 42
cleaned up


11

In [74]:
def add(a, b, c=None):
    if c is not None:
        raise MyError
    return a + b

In [75]:
add(1, 2, 3)

MyError: 

In [76]:
try:
    result = add(1, 2, 3)
except MyError as error:
    result = add(add(1, 2), 3)
result

6